In [ ]:
! pip install matplotlib
! pip install seaborn
! pip install tabulate
! pip install altair
! pip install plotly

In [5]:
import matplotlib.pyplot as plt
from data import load_data, save_to_json, load_from_json
from matplotlib.gridspec import GridSpec
import seaborn as sns
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tabulate import tabulate
import pandas as pd
from itertools import zip_longest
from pandas import MultiIndex
import altair as alt
import os

In [13]:
from math import log
# from scipy.stats import kendalltau

COLORS = ['red', 'blue', 'green', 'orange', 'purple', 'cyan', 'magenta', 'yellow', 'black', 'grey', 'pink']

def avg(l):
    return sum(l) / len(l)

def brier_score(pred):
    return avg([(x - 1) ** 2 for x in pred])

def log_loss(pred):
    return -1 * avg([log(x) for x in pred])

# def kendall_tau_for_results(model_results):
#     detection_scores = [i['detection_score'] for i in model_results] 
#     self_preferences = [i['self_preference'] for i in model_results]
    
#     return kendalltau(detection_scores, self_preferences).correlation

# def kendall_tau(x, y):
#     return kendalltau(x, y).correlation



In [10]:
models = results_to_load = ["gpt4_results_2_replace_50_sentence.json", "gpt4_results_5_replace_50_sentence.json"]
data_path = os.path.join("results","cnn","synonym")


# xsum_responses, xsum_articles, xsum_keys = load_data('xsum')
cnn_responses, cnn_articles, cnn_keys = load_data('cnn')

# xsum_results = {}
cnn_results = {}
for model in results_to_load:
    # xsum_results[model] = load_from_json(f'results/xsum/{model}_results.json')
    cnn_results[model] = load_from_json(os.path.join(data_path, model))
    


In [50]:
# loading old results for compairson for the selected keys
# all relevant keys
keys_list = [d['key'] for d in cnn_results['gpt4_results_5_replace_50_sentence.json']]
keys_list = set(keys_list)

# filter old results to keep only the 50 sentences
old_result = load_from_json(os.path.join("results","cnn","gpt4_results.json")) 
filtered_list = []
# Iterate over each dictionary in list_of_dicts
for d in old_result:
    # Check if any key in the current dictionary is in the keys_set
    if d['key'] in keys_list:
        filtered_list.append(d)

print(len(filtered_list))
print(len(cnn_results['gpt4_results_5_replace_50_sentence.json']))

# add to results
cnn_results['gpt4'] = filtered_list


200
200


In [53]:
models = models + ["gpt4"]

In [54]:
def print_zapped_table():
    table = [[model, 
                        avg([result['detection_score'] for result in cnn_results[model]]),
                        avg([0.5 if result['detection_score'] == 0.5 else 1 if result['detection_score'] > 0.5 else 0 for result in cnn_results[model]]),


    ] for model in models]

    print(tabulate(sorted(table, key = lambda x: x[-1]), headers = ['Model', 'Self-Rec (CNN)', 'Zapped' ]))
    # print(tabulate(sorted())

print_zapped_table()

Model                                      Self-Rec (CNN)    Zapped
---------------------------------------  ----------------  --------
gpt4_results_5_replace_50_sentence.json          0.371313     0.265
gpt4_results_2_replace_50_sentence.json          0.48937      0.505
gpt4                                             0.74366      0.885


In [19]:
# Main pairwise results
table = [[model, 
                    avg([result['detection_score'] for result in cnn_results[model]])
] for model in models]

table = [[row[0]] + [round(i, 3) for i in row[1:]] for row in table]
print(tabulate(table, headers = ['Model', 'Self-Rec']))

Model                                      Self-Rec
---------------------------------------  ----------
gpt4_results_2_replace_50_sentence.json       0.489
gpt4_results_5_replace_50_sentence.json       0.371


In [20]:
results = cnn_results
print(tabulate(sorted([[model, avg([result['detection_score'] for result in results[model]])]])))

---------------------------------------  --------
gpt4_results_5_replace_50_sentence.json  0.371313
---------------------------------------  --------


In [23]:
cnn_results['gpt4_results_5_replace_50_sentence.json']

[{'key': '8af047c2d83a91e8b745adfcaa7c282dfe1030a2',
  'model': 'human',
  'forward_detection': '1',
  'forward_detection_probability': 0.8105783935730584,
  'backward_detection': '1',
  'backward_detection_probability': 0.9213034906900109,
  'detection_score': 0.4446368879363054},
 {'key': '8af047c2d83a91e8b745adfcaa7c282dfe1030a2',
  'model': 'claude',
  'forward_detection': '1',
  'forward_detection_probability': 0.9675865390807186,
  'backward_detection': '1',
  'backward_detection_probability': 0.7686542349837918,
  'detection_score': 0.5994642167791562},
 {'key': '8af047c2d83a91e8b745adfcaa7c282dfe1030a2',
  'model': 'gpt35',
  'forward_detection': '1',
  'forward_detection_probability': 0.9546294197078434,
  'backward_detection': '1',
  'backward_detection_probability': 0.9688224635772926,
  'detection_score': 0.49290302178766476},
 {'key': '8af047c2d83a91e8b745adfcaa7c282dfe1030a2',
  'model': 'llama',
  'forward_detection': '1',
  'forward_detection_probability': 0.97752152783

In [25]:
keys_list = [d['key'] for d in cnn_results['gpt4_results_5_replace_50_sentence.json']]
print(keys_list)

['8af047c2d83a91e8b745adfcaa7c282dfe1030a2', '8af047c2d83a91e8b745adfcaa7c282dfe1030a2', '8af047c2d83a91e8b745adfcaa7c282dfe1030a2', '8af047c2d83a91e8b745adfcaa7c282dfe1030a2', 'dfcacd955a9168b2cf599c57a539dbfce43ea2a9', 'dfcacd955a9168b2cf599c57a539dbfce43ea2a9', 'dfcacd955a9168b2cf599c57a539dbfce43ea2a9', 'dfcacd955a9168b2cf599c57a539dbfce43ea2a9', 'f2463954bc7215d8a1f98cf0d85f280fd2affa58', 'f2463954bc7215d8a1f98cf0d85f280fd2affa58', 'f2463954bc7215d8a1f98cf0d85f280fd2affa58', 'f2463954bc7215d8a1f98cf0d85f280fd2affa58', '0e7a92a6f06e94cdd613860d4a3f4b92a06a2b43', '0e7a92a6f06e94cdd613860d4a3f4b92a06a2b43', '0e7a92a6f06e94cdd613860d4a3f4b92a06a2b43', '0e7a92a6f06e94cdd613860d4a3f4b92a06a2b43', '2905a15c5947042e42f4b52dc201d24822af20f1', '2905a15c5947042e42f4b52dc201d24822af20f1', '2905a15c5947042e42f4b52dc201d24822af20f1', '2905a15c5947042e42f4b52dc201d24822af20f1', '808a3d788317af05686ee71a71d2fbc5c19267bc', '808a3d788317af05686ee71a71d2fbc5c19267bc', '808a3d788317af05686ee71a71d2fb

In [ ]:
# Print table showing the kendall_tau_for_result for cn and xsum on each main model
print(tabulate(sorted([[model, avg([result['detection_score'] for result in xsum_results[model]]), avg([result['self_preference'] for result in xsum_results[model]]), avg([result['detection_score'] for result in cnn_results[model]]), avg([result['self_preference'] for result in cnn_results[model]]),kendall_tau_for_results(xsum_results[model]), kendall_tau_for_results(cnn_results[model])] for model in models if 'llama' in model], key = lambda x:x[0]), headers=['Model', 'Self-Rec [X]', 'Self-Pref [X]', 'Self-Rec [C]', 'Self-Pref [C]', 'XSUM T', 'CNN T']))

In [ ]:
def show_scatterplots(results, include_ambiguous=True):
    num_models = len(results.keys())

    plt.figure(figsize=(num_models * 6, num_models * 2))
    colors = ['blue', 'green', 'red']

    for i, model in enumerate(results.keys()):
        if not include_ambiguous:
            detection_scores = [i['detection_score'] for i in results[model] if i['forward_comparison'] != i['backward_comparison']]
            self_preferences = [i['self_preference'] for i in results[model] if i['forward_comparison'] != i['backward_comparison']]
        else:
            detection_scores = [i['detection_score'] for i in results[model]]
            self_preferences = [i['self_preference'] for i in results[model]]
        
        plt.subplot(1, 3, i+1)
        plt.scatter(detection_scores, self_preferences, color=colors[i])
        plt.xlabel('Detection Score')
        plt.ylabel('Self-Preference')
        plt.title(MODEL_TO_STRING[model])

    plt.suptitle('Detection Score vs Self-Preference (Token Probability)', fontsize=16, y=1) 
    plt.tight_layout()
    plt.show()

In [ ]:
def show_scatterplots(results, include_ambiguous=True):
    num_models = len(results.keys())

    plt.figure(figsize=(num_models * 6, num_models * 2))
    colors = ['blue', 'green', 'red']

    for i, model in enumerate(results.keys()):
        if not include_ambiguous:
            detection_scores = [i['detection_score'] for i in results[model] if i['forward_comparison'] != i['backward_comparison']]
            self_preferences = [i['self_preference'] for i in results[model] if i['forward_comparison'] != i['backward_comparison']]
        else:
            detection_scores = [i['detection_score'] for i in results[model]]
            self_preferences = [i['self_preference'] for i in results[model]]
        
        plt.subplot(1, 3, i+1)
        plt.scatter(detection_scores, self_preferences, color=colors[i])
        plt.xlabel('Detection Score')
        plt.ylabel('Self-Preference')
        plt.title(MODEL_TO_STRING[model])

    plt.suptitle('Detection Score vs Self-Preference (Token Probability)', fontsize=16, y=1) 
    plt.tight_layout()
    plt.show()

In [ ]:
def show_scatterplots_with_marginals(results, include_ambiguous=True):
    def is_valid(item):
        return 'detection_score' in item and 'self_preference' in item 

    for i, model in enumerate(results.keys()):
        if not include_ambiguous:
            detection_scores = [item['detection_score'] for item in results[model] if is_valid(item) and item['forward_comparison'] != item['backward_comparison'] and item['forward_detection'] != item['backward_detection']]
            self_preferences = [item['self_preference'] for item in results[model] if is_valid(item) and item['forward_comparison'] != item['backward_comparison'] and item['forward_detection'] != item['backward_detection']]
        else:
            detection_scores = [item['detection_score'] for item in results[model] if is_valid(item)]
            self_preferences = [item['self_preference'] for item in results[model] if is_valid(item)]

        # Create a jointplot for each model
        joint_plot = sns.jointplot(x=detection_scores, y=self_preferences, kind="scatter", color=COLORS[i % len(COLORS)], marginal_kws=dict(bins=15, fill=True))

        joint_plot.ax_joint.set_xlim(0, 1.0)
        joint_plot.ax_joint.set_ylim(0, 1.0)

        # Adjust the title position and font size
        joint_plot.fig.suptitle(f'{MODEL_TO_STRING[model]}', fontsize=14, y=1.05)
        # joint_plot.fig.suptitle(f'Detection Score vs Self-Preference (Token Probability) for {MODEL_TO_STRING[model]}', fontsize=14, y=1.05)

        # Adjust axis labels font size
        joint_plot.set_axis_labels('Detection Score', 'Self-Preference', fontsize=12)

        # Show the plot
        plt.savefig(f'plots/scatterplots/xsum/{model}.png', bbox_inches='tight')
        plt.show()


In [ ]:
show_scatterplots(results)
show_scatterplots(results, include_ambiguous=False)

In [ ]:
def show_heatmaps(results, include_ambiguous=True):
    num_models = len(results.keys())

    plt.figure(figsize=(num_models * 6, num_models * 2))
    colors = ['blue', 'green', 'red', 'orange']

    for i, model in enumerate(results.keys()):
        if not include_ambiguous:
            detection_scores = [i['detection_score'] for i in results[model] if i['forward_comparison'] != i['backward_comparison']]
            self_preferences = [i['self_preference'] for i in results[model] if i['forward_comparison'] != i['backward_comparison']]
        else:
            detection_scores = [i['detection_score'] for i in results[model]]
            self_preferences = [i['self_preference'] for i in results[model]]
        
        plt.subplot(1, num_models, i+1)
        plt.hexbin(detection_scores, self_preferences, gridsize=30, cmap='Blues')
        plt.colorbar(label='Density')
        plt.xlabel('Detection Score')
        plt.ylabel('Self-Preference')
        plt.title(MODEL_TO_STRING[model])

    plt.suptitle('Detection Score vs Self-Preference (Token Probability)', fontsize=16, y=1) 
    plt.tight_layout()
    plt.show()

In [ ]:
show_heatmaps(results)
show_heatmaps(results, include_ambiguous=False)

In [71]:
def plot_detection_score_vs_correlation(*results_dicts):
    # Extract 'a' and 'b' values
    x_values = []
    y_values = []
    plot_models = []
    d = {}
    for results in results_dicts:
        keys = [model for model in models if any(s in model for s in ['_2_ft_', '_10_ft_', '_500_ft_'])] #list(results.keys())
        plot_models += keys
        x_values += [avg([i['detection_score'] for i in results[key] if 'detection_score' in i]) for key in keys]
        y_values += [avg([i['self_preference'] for i in results[key] if 'self_preference' in i]) for key in keys]
        y_values += [kendall_tau_for_results(results[key]) for key in keys]
        for key in keys:
            d[key] = [(i['detection_score'], i['self_preference']) for i in results[key] if 'detection_score' in i and 'self_preference' in i]

    save_to_json(d, 'xsum_plot_data.json')
    # Create a scatter plot
    plt.figure(figsize=(8, 8))

    # Generate a color map with a unique color for each point
    markers = (['o'] * 3 + ['^'] * 3) * 4
    colors = ['red'] * 12 + ['blue'] * 12
    # colors = ['red', 'blue', 'green', 'orange', 'yellow', 'purple', 'black', 'pink', 'grey'][:len(plot_models)]
    
    # Plot each point
    print(plot_models)
    for i, (a, b, color, marker) in enumerate(zip_longest(x_values, y_values, colors, markers)):
        plt.scatter(a, b, color=color, marker=marker, label=MODEL_TO_STRING[plot_models[i]])

    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.gca().set_aspect('equal', adjustable='box')
    
    # Create a legend below the plot in a vertical column
    plt.legend(title="Key", loc='upper center', bbox_to_anchor=(0.5, -0.15), fancybox=True, shadow=True, ncol=1)

    # Add grid, title, and axis labels
    plt.grid(True)
    # plt.title('Detection Score vs. Self-Preference')
    plt.xlabel('Self-Recognition Score')
    plt.ylabel("Self-Preference")

    # Show the plot
    plt.savefig(f'plots/xsum_scaling_law.png', bbox_inches='tight')
    plt.show()
    return zip(plot_models, x_values, y_values)